In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)

ratingsDataset = '/Users/davidkatilius/Documents/Raw_Datasets/IMDB/title.ratings.tsv'
titleDataset = '/Users/davidkatilius/Documents/Raw_Datasets/IMDB/title.basics.tsv'

dfTitles = pd.read_csv(titleDataset, sep='\t')
dfRatings = pd.read_csv(ratingsDataset, sep='\t')

/var/folders/vb/t35ftt3d30b0gf_9v37z7st80000gn/T/ipykernel_38878/892188170.py:7: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  dfTitles = pd.read_csv(titleDataset, sep='\t')


In [5]:
dfTitles.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [6]:
dfRatings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1901
1,tt0000002,5.9,254
2,tt0000003,6.5,1694
3,tt0000004,5.7,166
4,tt0000005,6.2,2511


In [3]:
dfMovies = dfTitles[(dfTitles['titleType'] == 'movie') & (dfTitles['startYear'] != '\\N')]
dfMovies.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,\N,45,Romance
498,tt0000502,movie,Bohemios,Bohemios,0,1905,\N,100,\N
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,\N,70,"Action,Adventure,Biography"
587,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,\N,90,Drama
610,tt0000615,movie,Robbery Under Arms,Robbery Under Arms,0,1907,\N,\N,Drama


In [4]:
dfMovies.shape

(535173, 9)

In [7]:
dfRatedMovies = pd.merge(dfMovies, dfRatings, on = "tconst", how = "inner")
dfRatedMovies.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,\N,45,Romance,5.3,198
1,tt0000502,movie,Bohemios,Bohemios,0,1905,\N,100,\N,4.5,14
2,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,\N,70,"Action,Adventure,Biography",6.0,778
3,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,\N,90,Drama,4.7,19
4,tt0000615,movie,Robbery Under Arms,Robbery Under Arms,0,1907,\N,\N,Drama,4.5,23


In [8]:
dfRatedMovies.shape

(279919, 11)

In [9]:
dfRated100Times = dfRatedMovies[(dfRatedMovies['numVotes'] >= 100)]
dfRated100Times.shape

(113213, 11)

In [12]:
print(dfRated100Times.titleType.unique())
print(dfRated100Times.endYear.unique())

['movie']
['\\N']


In [ ]:
dfRated100Times.pop('titleType')
dfRated100Times.pop('endYear')

In [ ]:
dfRated10